# DataCollatorForLanguageModeling_3

## 1. Import 및 라이브러리 설치

In [1]:
import gc
import torch
import sys

gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install rouge_score
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [3]:
# mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-11-22 05:19:33--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-11-22 05:19:34--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd6b01db1d17df69a7f1ec25ef9.dl.dropboxusercontent.com/cd/0/get/BxOfZx_T10p6kffYPzCqllejL1-jVPgtdJVYyYgy_S38H_-DEPX88Ufc7kvqNRSIePoVqgqTozQFYJduaBB-Y-iqOSbjaizqh4XNnVQloysGl1KZL5u_10L7LreGUhotYpAhMwS4-4nsSXJMlK16jI0k7lWkbn0LnhrC8OIHMVqxVMocf0_0_X0bngFPJGTzjUU/file?dl=1# [following]
--2022-11-22 05:19:34--  https://ucd6b01db1d17df69a7f1ec25ef9.dl.dropboxusercontent.com/cd/0/get/BxOfZx_T10p6

In [4]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
from datasets import Dataset



import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)

from transformers import RobertaTokenizerFast
from transformers import EncoderDecoderModel
from dataclasses import dataclass, field
from typing import Optional
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
from tabulate import tabulate
from konlpy.tag import Mecab

KeyboardInterrupt: 

## 2. 모델 및 데이터 불러오기

### 1) 모델 불러오기

In [ ]:
model_checkpoints = "/aiffel/aiffel/Korean_Conversation_Summary/MLM_pretrain_3ep_221121/checkpoint-33000"#"gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

### 2) 데이터 불러오기 및 확인

In [ ]:
train_textfile_path = "data/train_text.csv"
val_textfile_path = "data/val_text.csv"

with open(train_textfile_path, encoding="utf-8") as f:
            train_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]     

with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]


In [ ]:
del train_textlines[0]
del val_textlines[0]

In [ ]:
train_df = pd.DataFrame(zip(train_textlines), columns=['Text'])
val_df = pd.DataFrame(zip(val_textlines), columns=['Text'])

In [ ]:
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df.head()

## 3. 데이터 EDA

In [ ]:
mecab = Mecab()

def sentence_len_total(data):
    text_split_text = []
    # 반복문으로 Mecab 적용
    for text_sen in tqdm(data['Text'].iloc[range(0, len(data))]):
        text_split_text.append(mecab.morphs(text_sen))
    
    temp = pd.DataFrame(zip(text_split_text), columns=['Text'])
    
    # Mecab 적용 후 길이 출력
    text_len = temp.Text.map(len)
    
    # text_len 사분위수 구하기    
    text_Q1 = text_len.quantile(.25)
    text_Q3 = text_len.quantile(.75)
    text_IQR = text_Q3 - text_Q1
    text_Q2 = text_len.quantile(.5)
    text_Q4 = text_len.quantile(1)
    text_threshold_len_left = text_Q1 - (1.5 * text_IQR)
    text_threshold_len_right = text_Q3 + (1.5 * text_IQR)

    
    print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
    print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
    print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
    print('텍스트의 왼쪽 울타리 범위 : {}'. format(text_threshold_len_left),
         '텍스트의 오른쪽 울타리 범위 : {}'. format(text_threshold_len_right))
    print('text_Q1 = {}'.format(text_Q1), 'headlines_Q1 = {}'.format(text_Q1))
    print('text_Q3 = {}'.format(text_Q3), 'headlines_Q3 = {}'.format(text_Q3))
    print('text_IQR = {}'.format(text_IQR), 'headlines_IQR = {}'.format(text_IQR))
    print('text_Q2 = {}'.format(text_Q2), 'headlines_Q2 = {}'.format(text_Q2))
    print('text_Q4 = {}'.format(text_Q4), 'headlines_Q4 = {}'.format(text_Q4))

    
    plt.subplot(1,1,1)
    plt.boxplot(text_len)
    plt.title('text')
    plt.show()

    plt.title('text')
    plt.hist(text_len, bins = 40)
    plt.xlabel('length of samples')
    plt.ylabel('number of samples')
    plt.show()
    
    return data


In [ ]:
sentence_len_total(train_df)

In [ ]:
sentence_len_total(val_df)

In [ ]:
print(train_df.head())
print(val_df.head())

## 4. 데이터 토큰화

### 1) dataset으로 변환

In [ ]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

### 2) EDA 바탕으로 길이 지정

In [ ]:
max_input = 70# 5e128
max_target = 70 # 5e 128
batch_size = 4
ignore_index = -100

### 3) 토큰화 함수 구현 및 토큰화

In [ ]:
def add_ignored_data(inputs, max_len, ignore_index):
    if len(inputs) < max_len:
        pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:max_len]

    return inputs

In [ ]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []

    inputs = [dialogue for dialogue in data_to_process['Text']]
    model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

    for i in range(len(data_to_process['Text'])):
        label_id.append(tokenizer.encode(data_to_process['Text'][i]))  
    for i in range(len(data_to_process['Text'])):
        label_id[i].append(tokenizer.eos_token_id)
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))

    model_inputs['labels'] = label_ids

    return model_inputs

In [ ]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text'])

## 5. 학습을 진행하기 위한 준비

### 1) config 지정

In [ ]:

model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 70#128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 2
model.config.length_penalty = 2.0
model.config.num_beams = 2


### 2) rounge 함수 구현

In [ ]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     print("labels_ids",labels_ids)
#     print("labels_ids[labels_ids == -100]",labels_ids[labels_ids == -100])
#     print("tokenizer.pad_token_id",tokenizer.pad_token_id)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    rouge_outputL = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid
    

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
        
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4), 
        
        "rougeL_precision": round(rouge_outputL.precision, 4),
        "rougeL_recall": round(rouge_outputL.recall, 4),
        "rougeL_fmeasure": round(rouge_outputL.fmeasure, 4),
    }

### 3) arguments 지정

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="MLM_pretrain_basev2_total10ep",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=16,
    learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=500,
    save_total_limit=3,
    evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    load_best_model_at_end = True,

)

### 4) data_collator

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

### 5) train

In [ ]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

## 6. 학습 결과 확인

In [ ]:
with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
        
val_df = pd.DataFrame(zip(val_textlines), columns=['Text'])
val_df.reset_index(inplace=True, drop=True)

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, num_beams=2,no_repeat_ngram_size=2, max_length=128,
                            suppress_tokens= [234,23782,14338,240,199,198,161,116, 14338, 239], 
                             attention_mask=attention_mask, top_p=0.92)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)# 여기에 기본 kobart가져오기?
import random
from random import randrange
ck_num = len(val_data)
test_samples = val_data.select(range(0, ck_num, 1000))# 0, len(test_data), 200

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1] # 여기에 체크포인트 가져오기 
# 연구해봐야한다.

In [ ]:
for i in range(len(summaries_after_tuning)):
    print('idx_{} '.format(i))
    print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()  